# Environment Prep



In [ ]:
import os

## DATA

you only need to move the data once - it will be there between restarts

### tfrecords
the tutorial says you need 2 new files under the training_demo/annotations folder, named 
- test.record
- train.record

In [ ]:
import os

In [ ]:
!pwd

In [ ]:
!ls /home/ec2-user/SageMaker/tensorflow/workspace/training_demo/annotations

## Model
download the ResNet50 640x640 model  

copy the pipeline.config file to my* folder.   Then edit locally with gedit.

In [ ]:
# you should see checkpoint, pipeline.config, saved_model
!ls /home/ec2-user/SageMaker/tensorflow/workspace/training_demo/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/

In [ ]:
!ls /home/ec2-user/SageMaker/tensorflow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn -l

## TF 2.x GPU
Common Problems
- if list_physical_devices shows GPU and gpu_device_name is none, then you probably have a CUDA version issue - check CUDA version, PATH & LD_LIBRARY path in ./bashrc.     e.g.   path needs to include CUDA 10.1 for TF2.2
- check your version of TensorFlow, one of the steps automatically upgrades from TF 2.x to 2.4.   Thus, you need CUDA 11.0 installed.   However, I didn't notice any path requirements...

In [1]:
import tensorflow as tf

In [2]:
print (tf.__version__)

2.2.0


In [3]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [4]:
physical_devices= tf.config.experimental.list_physical_devices('GPU')
print(len(physical_devices))

1


In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6110618784829385700
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 12987503803283228944
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 2949021694671860497
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5368897856
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16719459921663931388
physical_device_desc: "device: 0, name: GeForce GTX 1060 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [6]:
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

tf.Tensor(-505.21414, shape=(), dtype=float32)


## Tutorial - Object Detection API Setup

In [ ]:
!pip install pycocotools

In [ ]:
import pycocotools

### API

commom problems:

#### ASUS


#### Dell Inspiron

```ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
paramiko 2.4.2 requires cryptography>=1.5, which is not installed.
fabric 2.4.0 requires cryptography>=1.1, which is not installed.
aws 0.2.5 requires boto, which is not installed.
tensorflow-gpu 2.2.0 requires tensorboard<2.3.0,>=2.2.0, but you have tensorboard 2.4.0 which is incompatible.
tensorflow-gpu 2.2.0 requires tensorflow-estimator<2.3.0,>=2.2.0, but you have tensorflow-estimator 2.4.0 which is incompatible.

```
- pip install crytography  # 3.3.1
- pip install boto # 2.49.0
- DON'T mess with tensorflow* dependencies - because this script is in some kind of loop!   I had 2.2 then it uninstalled and upgraded to 2.4 so the fact that it is complaining about 2.4 is kind of bogus.   I suppose you could back it down and it would probably work.  BUT - I did nothing and then reran the script and it worked fine

Successfully installed object-detection-0.1

In [ ]:
original_cwd = cwd = os.getcwd()
print ("Original working directory:", original_cwd)

In [ ]:
os.chdir('/home/ec2-user/SageMaker/tensorflow/models/research/')
cwd = os.getcwd() 
print("Current working directory is:", cwd)

In [ ]:
# make sure your kernel is correct:  conda_tensorflow2_p36
!python -m pip install .

### Test the API
you should be in models/research

Possible Problems:
- GPU error - memory;   you may need to close this notebook and others to free up GPU memory

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
os.chdir(original_cwd)
cwd = os.getcwd() 
print("back to original working directory:", cwd)

## train

speed benchmarks
- SageMaker - 27.7 minutes!! per step - that was CPU only becuase CUDA 11.0 is not installed
- ASUS 1080 TI - 0.7 sec (8 images/batch)
- Dell Inspiron 1060 - 1.0  (4 images/batch, memory warnings)

In [ ]:
!cp /home/ec2-user/SageMaker/tensorflow/models/research/object_detection/model_main_tf2.py .

In [ ]:
os.chdir('/home/ec2-user/SageMaker/tensorflow/workspace/training_demo/')
cwd = os.getcwd() 
print("Current working directory is:", cwd)

In [ ]:
!python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config